In [ ]:
# %pip install nba_api

## NBA API Experiments
While I was experimenting the `nba_api` I created this notebook. It's a bit of a mixed bag of tests but certainly a worthy inclusion of things you might try yourself.

There is no real order in this notebook -- feel free to submit a [pull request](https://github.com/codingforentrepreneurs/Try-Pandas/pulls) if you find ways to improve it.


#### _NBA Shot Chart explainer_
Interesting project (perhas a future video)?
- https://www.youtube.com/watch?v=a3u-3gEYvxM
- https://github.com/hkair/nba-shotcharts

In [ ]:
import pandas as pd
import pathlib

In [ ]:
from nba_api.stats.endpoints import commonplayerinfo, playercareerstats
from nba_api.stats.static import players

In [ ]:
BASE_DIR = pathlib.Path().resolve().parent
SAMPLES_DIR = BASE_DIR / "samples"
salary_df = pd.read_csv(SAMPLES_DIR / '5-player-adj-salaries-audit.csv')

In [ ]:
def split_player_name(row):
    name = row['player']
    name_list = name.split()
    row['first_name'] = None
    row['last_name'] = None
    if len(name_list) == 2:
        row['first_name'] = name_list[0]
        row['last_name'] = name_list[1]
    if len(name_list) > 2 :
        row['first_name'] = " ".join(name_list[:-1])
        row['last_name'] = name_list[-1]
    return row

salary_df = salary_df.apply(split_player_name, axis=1)
# salary_df.sort_values("player", inplace=True)
salary_df.head()

In [ ]:
career = playercareerstats.PlayerCareerStats(player_id=3)
career

In [ ]:
def season_id_to_season_end(val):
    season_start, season_end = val.split("-")
    if f"{season_start}".startswith("1"):
        season_end = f"19{season_end}"
    else:
        season_end = f"20{season_end}"
    return season_end

# Charles Barkle player_id = 3
def get_season_data(player_id=1):
    career = playercareerstats.PlayerCareerStats(player_id=player_id)
    player_df = career.get_data_frames()[0]
    player_df['season_start'] = player_df['SEASON_ID'].apply(lambda x: x.split("-")[0])
    player_df['season_end'] = player_df['SEASON_ID'].apply(season_id_to_season_end)
    # player_df_final = player_df.copy()[['PLAYER_ID', 'TEAM_ABBREVIATION', 'season_start', 'season_end']]
    return player_df

In [ ]:
player_ = get_season_data(player_id=3)
player_.head()

In [ ]:
nba_players = players.get_players()

nba_players_df = pd.DataFrame(nba_players)
nba_players_df.head()

In [ ]:
# this is not efficient, perhaps another method to enrich all the data.

# all_player_teams = []
# for p_id in nba_players_df['id'].values[:10]:
#     all_player_teams.append(get_season_data(player_id=p_id))

# all_player_teams_df = pd.concat(all_player_teams)

In [ ]:
salary_df.merge(nba_players_df, left_on="player", right_on="full_name")

In [ ]:
from nba_api.stats.static import teams

nba_teams = teams.get_teams()
# Select the dictionary for the Celtics, which contains their team ID
celtics = [team for team in nba_teams if team['abbreviation'] == 'BOS'][0]
celtics_id = celtics['id']

In [ ]:
from nba_api.stats.endpoints import leaguegamefinder

# Query for games where the Celtics were playing
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=celtics_id)
# The first DataFrame of those returned is what we want.
games = gamefinder.get_data_frames()[0]
games.head()

In [ ]:
player_info = commonplayerinfo.CommonPlayerInfo(player_id=51)
player_info_df = pd.concat(player_info.get_data_frames()[:1])
player_info_df.head()

In [ ]:
from nba_api.stats.endpoints import playercareerstats
# Anthony Davis
career = playercareerstats.PlayerCareerStats(player_id=51)
player_df = career.get_data_frames()[0]
player_df['season_start'] = player_df['SEASON_ID'].apply(lambda x: x.split("-")[0])

def season_end(val):
    season_start, season_end = val.split("-")
    if f"{season_end}".startswith("9"):
        season_end = f"19{season_end}"
    else:
        season_end = f"20{season_end}"
    return season_end

player_df['season_end'] = player_df['SEASON_ID'].apply(season_end)
player_df_final = player_df.copy()[['PLAYER_ID', 'TEAM_ABBREVIATION', 'season_start', 'season_end']]
player_df_final.tail(n=100)

In [ ]:
player_df.columns

In [ ]:
player_games = player_df.copy()[['SEASON_ID', 'GAME_ID', "TEAM_ID", 'TEAM_NAME', "WL", "MIN", "PTS"]]
player_games.head()

In [ ]:
players = players.get_players()

In [ ]:
players_df = pd.DataFrame(players)

In [ ]:
players_df.head()

In [ ]:


player_info.common_player_info.get_data_frame()

In [ ]:
# def add_common_info(row):
#     player_id = row['id']
#     player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
#     print(player_info)
#     return row

# players_df.apply(add_common_info , axis=1)
# players_df.head()

In [ ]:
from nba_api.stats.static import teams


teams = teams.get_teams()

In [ ]:
teams_df = pd.DataFrame(teams)
teams_df.head()